# Joint estimation of parameters

Iterative procedure to jointly fit MCAR drift parameter $\mathbf{A}$ and driving Levy parameters $(b, \Sigma)$ from $\mathbb{Y}_{\mathcal{P}} = \{\mathbf{Y}_s,\ s\in\mathcal{P}\}$.

We start by fixing $\mathbf{\hat{b}} = \mathbf{0}$ (note the choice of $\Sigma$ does not affect the estimation of $\mathbf{A}$) and:
1. Estimate $\mathbf{\hat{A}}$ using $\mathbf{\hat{b}}$;


2. Recover the increments of the background driving Levy process $\{\Delta \mathbf{L}_s (\mathbf{\hat{A}}),\ s\in\mathcal{P}\}$ using $\mathbf{\hat{A}}$ and estimate:
    
    
    - $\mathbf{\mu} := \mathbb{E}[\mathbf{L}_1]$ from $\{\Delta \mathbf{L}_s,\ s\in\mathcal{P}\}$ either by setting $\mathbf{hat{\mu}} = t^{-1} \mathbf{L}_t (\mathbf{\hat{A}})$ or $\mathbf{\hat{\mu}} = \hat{A}_p \left(\frac{1}{|\mathcal{P}|} \sum_{s\in\mathcal{P}} \mathbf{Y}_s\right)$;
    
    
    - $\Sigma$ and $\displaystyle \mathbf{f}:=\int_{\|\mathbf{z}\|>1} \mathbf{z} F(d\mathbf{z})$ from $\{\Delta \mathbf{L}_s,\ s\in\mathcal{P}\}$ by applying (Gegler, 2011);
    
    
    - set $\mathbf{\hat{b}} = \mathbf{\hat\mu} - \mathbf{\hat{f}}$.


3. Repeat 1 and 2 until convergence criterion is met.

Note to estimate $\mathbf{\mu}$ we require $t$ (time horizon) to be quite large as this relies on the statistic $t^{-1} \mathbf{L}_t$ (where $\mathbf{L}_t$ is recovered...). Might be better to use a "method of moments estimator" relying on the mean of $\mathbb{Y}$ and $\mathbf{A}$ to estimate $\mathbf{\mu}$.

In the general case of a stationary MCAR process we have that the state space mean is
$$ \mathbb{E}[\mathbf{X}_s] = - \mathcal{A}_{\mathbf{A}}^{-1} \mathcal{E} \mathbf{\mu}, $$
which implies $\mathbb{E}[D^j\mathbf{Y}_s] = 0$ for $j=1, \ldots, p-1$ and $\mathbb{E}[\mathbf{Y}_s] = {A}_p^{-1} \mathbf{\mu}$. Thus we can estimate
$$\mathbf{\hat{\mu}} = \hat{A}_p \left(\frac{1}{|\mathcal{P}|} \sum_{s\in\mathcal{P}} \mathbf{Y}_s\right).$$

The above procedure can be modified when some of the parameters ($\mathbf{A}, \mathbf{b}, \Sigma$ or $\mathbf{f}$) are known/we have strong priors on their values by simply keeping them fixed.

In the following,
$$ \mathbf{\mu} = \mathbb{E}[\mathbf{L}_1] = \mathbf{b} + \int_{\|\mathbf{z}\|>1} \mathbf{z} F(d\mathbf{z}) = \mathbf{\tilde{b}} + \int_{\mathbb{R}^d} \mathbf{z} F(d\mathbf{z}),$$
thus 
$$ \mathbf{\tilde{b}} = \mathbf{b} - \int_{\|\mathbf{z}\|\leq 1} \mathbf{z} F(d\mathbf{z}).$$

where $\mathbf{b}$ is the drift parameter with truncation function $\tau(z) = 1_{\{\|z\|\leq 1\}}$, $\mathbf{\tilde{b}}$ is the drift parameter with truncation function $\tau(z) \equiv 0$ and $\mathbf{\mu}$ is the drift parameter with truncation function $\tau(z) \equiv 1$.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from simulate_mcar import MCAR_A, gamma_increments, asymm_gamma_increments, simulate_MCAR_stat_distr_approx, simulate_MCAR_approx, simulate_MCAR_compound_poisson, simulate_MCAR_stat_distr_compound_poisson
from estimate_mcar import estimate_MCAR
import numpy as np
import scipy
from functools import partial

## OU process

In [21]:
# OU parameters
d = 1
p = 2

# build state space autoregressive matrix
AA = [np.array([[1., 0.], [0., 2.]])]
AA= [np.array([[1.]]), np.array([[2.]])]
A_AA = MCAR_A(AA)

# check eigenvalues are negative, i.e. AA is in int(\mathfak{A})
evals, evecs = np.linalg.eig(A_AA)
assert (evals < 0).all()

# parameters of Levy process
b = 0*np.ones(d)
Sigma = 2*np.eye(d)

# finest grid possible
T = 100
N = 100001
P = np.linspace(0, T, N)

In [23]:
# symmetric compound poisson process
rate = 10
jump_F = scipy.stats.multivariate_normal(mean=np.zeros(d), cov=np.eye(d))

np.random.seed(12345)
a = b
x0 = simulate_MCAR_stat_distr_compound_poisson(A_AA, a, Sigma, rate=rate, jump_F=jump_F)
Y = simulate_MCAR_compound_poisson(P, A_AA, x0, a, Sigma, rate=rate, jump_F=jump_F, uniform=True)

AA_hat, (a_hat, Sigma_hat) = estimate_MCAR(Y, p, P, P, jump_activity='finite')

np.set_printoptions(precision=2, floatmode='fixed')
print(f'AA = \n{AA}')
print(f'AA_hat = \n{AA_hat}')
print('------------------------')
print(f'a = {a}')
print(f'a_hat = {a_hat}')
print('------------------------')
print(f'Sigma = \n{Sigma}')
print(f'Sigma_hat = \n{Sigma_hat}')

AA = 
[array([[1.00]]), array([[2.00]])]
AA_hat = 
[array([[0.80]]), array([[2.11]])]
------------------------
a = [0.00]
a_hat = [-0.02]
------------------------
Sigma = 
[[2.00]]
Sigma_hat = 
[[1.39]]


In [274]:
class point_jumps():
    def __init__(self, d, jump_size):
        self.d = d
        self.jump_size = jump_size

    def rvs(self, size):
        return self.jump_size * np.ones((size, self.d))
    
# asymmetric compound poisson process 
rate = 2
jump_size = 2
jump_F = point_jumps(d, jump_size)

np.random.seed(123)
a = b - rate * jump_size * (np.abs(jump_size) < 1)
x0 = simulate_MCAR_stat_distr_compound_poisson(A_AA, a, Sigma, rate=rate, jump_F=jump_F)
Y = simulate_MCAR_compound_poisson(P, A_AA, x0, a, Sigma, rate=rate, jump_F=jump_F, uniform=True)

AA_hat, (a_hat, Sigma_hat) = estimate_MCAR(Y, p, P, P, jump_activity='finite')

np.set_printoptions(precision=2, floatmode='fixed')
print(f'AA = \n{AA[0]}')
print(f'AA_hat = \n{AA_hat[0]}')
print('------------------------')
print(f'a = {a}')
print(f'a_hat = {a_hat}')
print('------------------------')
print(f'Sigma = \n{Sigma}')
print(f'Sigma_hat = \n{Sigma_hat}')

AA = 
[[1.00 0.00]
 [0.00 2.00]]
AA_hat = 
[[ 0.94 -0.01]
 [-0.07  1.90]]
------------------------
a = [1.00 1.00]
a_hat = [0.71 0.48]
------------------------
Sigma = 
[[2.00 0.00]
 [0.00 2.00]]
Sigma_hat = 
[[1.99 0.00]
 [0.00 1.99]]


In [275]:
# symmetric Gamma increments
jumps = partial(gamma_increments, shape=5, scale=5, d=d)

np.random.seed(123)
a = b
x0 = simulate_MCAR_stat_distr_approx(A_AA, a, Sigma, jumps)
Y = simulate_MCAR_approx(P, A_AA, x0, a, Sigma, jumps, uniform=True)

AA_hat, (b_hat, Sigma_hat) = estimate_MCAR(Y, p, P, P, jump_activity = 'infinite')

np.set_printoptions(precision=2, floatmode='fixed')
print(f'AA = \n{AA[0]}')
print(f'AA_hat = \n{AA_hat[0]}')
print('------------------------')
print(f'b = {b}')
print(f'b_hat = {b_hat}')
print('------------------------')
print(f'Sigma = \n{Sigma}')
print(f'Sigma_hat = \n{Sigma_hat}')

AA = 
[[1.00 0.00]
 [0.00 2.00]]
AA_hat = 
[[ 9.59e-01  1.08e-03]
 [-1.08e-02  1.92e+00]]
------------------------
b = [1.00 1.00]
b_hat = [1.35 1.33]
------------------------
Sigma = 
[[2.00 0.00]
 [0.00 2.00]]
Sigma_hat = 
[[ 2.21 -0.00]
 [-0.00  2.21]]


In [280]:
# asymmetric Gamma increments
jumps = partial(asymm_gamma_increments, shape=1, scale=1, d=d)

np.random.seed(123)
a = b - (1 - np.exp(-1))
x0 = simulate_MCAR_stat_distr_approx(A_AA, a, Sigma, jumps)
Y = simulate_MCAR_approx(P, A_AA, x0, a, Sigma, jumps, uniform=True)

AA_hat, (b_hat, Sigma_hat) = estimate_MCAR(Y, p, P, P, jump_activity='infinite')

np.set_printoptions(precision=2, floatmode='fixed')
print(f'AA = \n{AA[0]}')
print(f'AA_hat = \n{AA_hat[0]}')
print('------------------------')
print(f'b = {b}')
print(f'b_hat = {b_hat}')
print('------------------------')
print(f'Sigma = \n{Sigma}')
print(f'Sigma_hat = \n{Sigma_hat}')

AA = 
[[1.00 0.00]
 [0.00 1.00]]
AA_hat = 
[[1.21 0.53]
 [0.33 1.36]]
------------------------
b = [1.00 1.00]
b_hat = [2.38 2.38]
------------------------
Sigma = 
[[2.00 0.00]
 [0.00 2.00]]
Sigma_hat = 
[[ 2.01e+00 -1.60e-03]
 [-1.60e-03  2.02e+00]]
